# Model Save - CNN (AE Min 8 Node)

### Import List

In [1]:
import matplotlib.pyplot as plt
import mglearn
import numpy as np
import os
import random

C:\Users\KSY\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
C:\Users\KSY\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


### Method Definition

In [2]:
base_dir = "..\\..\\..\\data\\BLE_data\\"
def read_test_data(set_num, model_name, mode):
    if model_name == 5:
        raw_data = np.loadtxt(base_dir + "raw_data\\in_" + mode + "_" + str(set_num) + '.csv', delimiter=',', dtype=np.float32)
    elif model_name == 6:
        raw_data = np.loadtxt(base_dir + "raw_data\\out_" + mode + "_" + str(set_num) + '.csv', delimiter=',', dtype=np.float32)
    b_data    = raw_data[:,0:3]
    cell_data = raw_data[:,3:13]

    return b_data, cell_data

def read_denoised_test_data(set_num, model_name, mode):
    if model_name == 5:
        raw_data = np.loadtxt(base_dir + "filtered_data\\in_" + mode + "_" + str(set_num) + '_denoised.csv', delimiter=',', dtype=np.float32)
    elif model_name == 6:
        raw_data = np.loadtxt(base_dir + "filtered_data\\out_" + mode + "_" + str(set_num) + '_denoised.csv', delimiter=',', dtype=np.float32)
    b_data    = raw_data[:,0:3]
    cell_data = raw_data[:,3:13]

    return b_data, cell_data

def make_plot(y, num, begin, end):  
    fig = plt.figure(figsize=(15, 5))
    fig.patch.set_facecolor('xkcd:white')
    x = range(len(y))
    plt.bar(x, y, color="blue")
    plt.axis([0, num, begin, end])
    plt.show()

## Start

In [3]:
# model_name: 5 (car_in) / 6 (car_out)
# Car-in data
model_name = 5
set_num = 6000
train_set_num = int(set_num * 0.75)
test_set_num = int(set_num * 0.25)

x_in_train_noisy, y_in_train_noisy = read_test_data(train_set_num, model_name, "train")
x_in_train_denoised, y_in_train_denoised = read_denoised_test_data(train_set_num, model_name, "train")
x_in_test_noisy, y_in_test_noisy = read_test_data(test_set_num, model_name, "test")
x_in_test_denoised, y_in_test_denoised = read_denoised_test_data(test_set_num, model_name, "test")

In [4]:
# model_name: 5 (car_in) / 6 (car_out)
# Car-out data
model_name = 6
set_num = 6000
train_set_num = int(set_num * 0.75)
test_set_num = int(set_num * 0.25)

x_out_train_noisy, y_out_train_noisy = read_test_data(train_set_num, model_name, "train")
x_out_train_denoised, y_out_train_denoised = read_denoised_test_data(train_set_num, model_name, "train")
x_out_test_noisy, y_out_test_noisy = read_test_data(test_set_num, model_name, "test")
x_out_test_denoised, y_out_test_denoised = read_denoised_test_data(test_set_num, model_name, "test")

In [5]:
# Car-in
ae_x_in_train_noisy = x_in_train_noisy.astype('float32') / -100.
ae_x_in_train_denoised = x_in_train_denoised.astype('float32') / -100.
ae_x_in_test_noisy = x_in_test_noisy.astype('float32') / -100.
ae_x_in_test_denoised = x_in_test_denoised.astype('float32') / -100.

In [6]:
# Car-out
ae_x_out_train_noisy = x_out_train_noisy.astype('float32') / -100.
ae_x_out_train_denoised = x_out_train_denoised.astype('float32') / -100.
ae_x_out_test_noisy = x_out_test_noisy.astype('float32') / -100.
ae_x_out_test_denoised = x_out_test_denoised.astype('float32') / -100.

## Autoencoder - Denoising (Car in)

In [7]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.models import load_model

autoencoder_01 = load_model('autoencoder_01.h5')

Using TensorFlow backend.


## Autoencoder - Denoising (Car out)

In [8]:
autoencoder_02 = load_model('autoencoder_02.h5')

## Autoencoder - Converting

## Description
- Autoencoder
    - Car in (noisy)를 Autoencoder에 넣은 결과 Car in (denoised)
    - Car in (denoised) 입력
    - Car out (denoised) 정답

In [9]:
autoencoder_03 = load_model('autoencoder_03.h5')

## Autoencoder - Converting

## Description
- Autoencoder
    - Car in (denoised) 입력
    - Car out (denoised) 정답

In [10]:
autoencoder_04 = load_model('autoencoder_04.h5')

## Test with CNN Model

In [11]:
from keras.layers.convolutional import Conv1D
from keras.layers import GlobalAveragePooling1D
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.models import Sequential

# Description
- cnn_01
    - Car in (noisy)를 AE_03에 넣으면 결과 Car out (denoised = converted)
    - Car out (denoised = converted)를 입력 (Input)
- cnn_02
    - Car in (filtered)를 AE_04에 넣어서 결과 Car out (denoised)
    - Car out (denoised)를 입력 (Input)
- cnn_03
    - Car out (noisy)를 AE_02에 넣으면 Car out (denoised)
    - Car out (denoised)를 입력 (Input)
- cnn_04
    - Car out (filtered)를 입력 (Input)
- Output (정답)
    - Car out (filtered)의 레이블

## CNN_01

In [12]:
after_autoencoder_x_train = autoencoder_03.predict(ae_x_in_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_02.predict(ae_x_in_test_noisy) * (-100)

x_data, y_data = after_autoencoder_x_train, y_out_train_denoised
x_data_test, y_data_test = after_autoencoder_x_test, y_out_test_denoised

# numpy 행렬로 변환 (행, 열, 1차원)
x_data = x_data.reshape(45000, 3, 1)
x_data_test = x_data_test.reshape(15000, 3, 1)

cnn_01 = Sequential()
cnn_01.add(Conv1D(128,3,activation='sigmoid', input_shape = (3,1), padding = 'same', strides = 1))
cnn_01.add(GlobalAveragePooling1D())  # 각 차원마다 평균값 도출
cnn_01.add(Dropout(0.5))  # 비활성화(0.1), 활성화(0.9)
cnn_01.add(Dense(10, activation = 'sigmoid'))  # 신경망 비슷

# CNN training
cnn_01.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
cnn_01.fit(x_data, y_data, batch_size = 100, epochs = 200)

train_result = cnn_01.evaluate(x_data, y_data, batch_size = 1000, verbose = 1)
test_result = cnn_01.evaluate(x_data_test, y_data_test, batch_size = 1000, verbose = 1)

# Save model
cnn_01.save('cnn_01.h5')

# CNN accuracy
print("[CNN] Training data accuracy: {:.4f}".format(train_result[1]))
print("[CNN] Testing data accuracy: {:.4f}".format(test_result[1]))

Epoch 1/200
45000/45000 [==============================] - 1s 22us/step - loss: 0.3469 - accuracy: 0.8924
Epoch 2/200
45000/45000 [==============================] - 1s 20us/step - loss: 0.3290 - accuracy: 0.9000
Epoch 3/200
45000/45000 [==============================] - 1s 20us/step - loss: 0.3276 - accuracy: 0.9000
Epoch 4/200
45000/45000 [==============================] - 1s 21us/step - loss: 0.3268 - accuracy: 0.9000
Epoch 5/200
45000/45000 [==============================] - 1s 20us/step - loss: 0.3262 - accuracy: 0.9000
Epoch 6/200
45000/45000 [==============================] - 1s 20us/step - loss: 0.3257 - accuracy: 0.9000
Epoch 7/200
45000/45000 [==============================] - 1s 22us/step - loss: 0.3252 - accuracy: 0.9000
Epoch 8/200
45000/45000 [==============================] - 1s 21us/step - loss: 0.3247 - accuracy: 0.9000
Epoch 9/200
45000/45000 [==============================] - 1s 23us/step - loss: 0.3239 - accuracy: 0.9000
Epoch 10/200
45000/45000 [====================

45000/45000 [==============================] - 2s 35us/step - loss: 0.3040 - accuracy: 0.9002
Epoch 154/200
45000/45000 [==============================] - 1s 28us/step - loss: 0.3039 - accuracy: 0.9005
Epoch 155/200
45000/45000 [==============================] - 1s 29us/step - loss: 0.3039 - accuracy: 0.9004
Epoch 156/200
45000/45000 [==============================] - 1s 26us/step - loss: 0.3040 - accuracy: 0.9003
Epoch 157/200
45000/45000 [==============================] - 1s 26us/step - loss: 0.3040 - accuracy: 0.9004
Epoch 158/200
45000/45000 [==============================] - 1s 24us/step - loss: 0.3038 - accuracy: 0.9004
Epoch 159/200
45000/45000 [==============================] - 1s 25us/step - loss: 0.3037 - accuracy: 0.9004
Epoch 160/200
45000/45000 [==============================] - 1s 28us/step - loss: 0.3037 - accuracy: 0.9003
Epoch 161/200
45000/45000 [==============================] - 1s 26us/step - loss: 0.3037 - accuracy: 0.9003
Epoch 162/200
45000/45000 [===============

## CNN_02

In [13]:
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train_denoised) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test_denoised) * (-100)

x_data, y_data = after_autoencoder_x_train, y_out_train_denoised
x_data_test, y_data_test = after_autoencoder_x_test, y_out_test_denoised

# numpy 행렬로 변환 (행, 열, 1차원)
x_data = x_data.reshape(45000, 3, 1)
x_data_test = x_data_test.reshape(15000, 3, 1)

cnn_02 = Sequential()
cnn_02.add(Conv1D(128,3,activation='sigmoid', input_shape = (3,1), padding = 'same', strides = 1))
cnn_02.add(GlobalAveragePooling1D())  # 각 차원마다 평균값 도출
cnn_02.add(Dropout(0.5))  # 비활성화(0.1), 활성화(0.9)
cnn_02.add(Dense(10, activation = 'sigmoid'))  # 신경망 비슷

# CNN training
cnn_02.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
cnn_02.fit(x_data, y_data, batch_size = 100, epochs = 200)

train_result = cnn_02.evaluate(x_data, y_data, batch_size = 1000, verbose = 1)
test_result = cnn_02.evaluate(x_data_test, y_data_test, batch_size = 1000, verbose = 1)

# Save model
cnn_02.save('cnn_02.h5')

# CNN accuracy
print("[CNN] Training data accuracy: {:.4f}".format(train_result[1]))
print("[CNN] Testing data accuracy: {:.4f}".format(test_result[1]))

Epoch 1/200
45000/45000 [==============================] - 1s 24us/step - loss: 0.3029 - accuracy: 0.8961
Epoch 2/200
45000/45000 [==============================] - 1s 20us/step - loss: 0.2337 - accuracy: 0.9014
Epoch 3/200
45000/45000 [==============================] - 1s 19us/step - loss: 0.2093 - accuracy: 0.9039
Epoch 4/200
45000/45000 [==============================] - 1s 22us/step - loss: 0.1929 - accuracy: 0.9084
Epoch 5/200
45000/45000 [==============================] - 1s 20us/step - loss: 0.1786 - accuracy: 0.9150
Epoch 6/200
45000/45000 [==============================] - 1s 23us/step - loss: 0.1662 - accuracy: 0.9215
Epoch 7/200
45000/45000 [==============================] - 1s 18us/step - loss: 0.1558 - accuracy: 0.9268
Epoch 8/200
45000/45000 [==============================] - 1s 18us/step - loss: 0.1464 - accuracy: 0.9321
Epoch 9/200
45000/45000 [==============================] - 1s 20us/step - loss: 0.1383 - accuracy: 0.9361
Epoch 10/200
45000/45000 [====================

45000/45000 [==============================] - 1s 19us/step - loss: 0.0525 - accuracy: 0.9838
Epoch 154/200
45000/45000 [==============================] - 1s 20us/step - loss: 0.0524 - accuracy: 0.9835
Epoch 155/200
45000/45000 [==============================] - 1s 19us/step - loss: 0.0529 - accuracy: 0.9836
Epoch 156/200
45000/45000 [==============================] - 1s 18us/step - loss: 0.0523 - accuracy: 0.9839
Epoch 157/200
45000/45000 [==============================] - 1s 18us/step - loss: 0.0526 - accuracy: 0.9838
Epoch 158/200
45000/45000 [==============================] - 1s 31us/step - loss: 0.0521 - accuracy: 0.9839
Epoch 159/200
45000/45000 [==============================] - 1s 23us/step - loss: 0.0521 - accuracy: 0.9839
Epoch 160/200
45000/45000 [==============================] - 1s 21us/step - loss: 0.0523 - accuracy: 0.9840
Epoch 161/200
45000/45000 [==============================] - 1s 29us/step - loss: 0.0520 - accuracy: 0.9839
Epoch 162/200
45000/45000 [===============

## CNN_03

In [14]:
after_autoencoder_x_train = autoencoder_02.predict(ae_x_out_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_02.predict(ae_x_out_test_noisy) * (-100)

x_data, y_data = after_autoencoder_x_train, y_out_train_denoised
x_data_test, y_data_test = after_autoencoder_x_test, y_out_test_denoised

# numpy 행렬로 변환 (행, 열, 1차원)
x_data = x_data.reshape(45000, 3, 1)
x_data_test = x_data_test.reshape(15000, 3, 1)

cnn_03 = Sequential()
cnn_03.add(Conv1D(128,3,activation='sigmoid', input_shape = (3,1), padding = 'same', strides = 1))
cnn_03.add(GlobalAveragePooling1D())  # 각 차원마다 평균값 도출
cnn_03.add(Dropout(0.5))  # 비활성화(0.1), 활성화(0.9)
cnn_03.add(Dense(10, activation = 'sigmoid'))  # 신경망 비슷

# CNN training
cnn_03.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
cnn_03.fit(x_data, y_data, batch_size = 100, epochs = 200)

train_result = cnn_03.evaluate(x_data, y_data, batch_size = 1000, verbose = 1)
test_result = cnn_03.evaluate(x_data_test, y_data_test, batch_size = 1000, verbose = 1)

# Save model
cnn_03.save('cnn_03.h5')

# CNN accuracy
print("[CNN] Training data accuracy: {:.4f}".format(train_result[1]))
print("[CNN] Testing data accuracy: {:.4f}".format(test_result[1]))

Epoch 1/200
45000/45000 [==============================] - 1s 22us/step - loss: 0.2990 - accuracy: 0.8951
Epoch 2/200
45000/45000 [==============================] - 1s 20us/step - loss: 0.2250 - accuracy: 0.9015
Epoch 3/200
45000/45000 [==============================] - 1s 19us/step - loss: 0.1988 - accuracy: 0.9050
Epoch 4/200
45000/45000 [==============================] - 1s 24us/step - loss: 0.1797 - accuracy: 0.9113
Epoch 5/200
45000/45000 [==============================] - 1s 32us/step - loss: 0.1639 - accuracy: 0.9187
Epoch 6/200
45000/45000 [==============================] - 1s 26us/step - loss: 0.1503 - accuracy: 0.9262
Epoch 7/200
45000/45000 [==============================] - 1s 25us/step - loss: 0.1384 - accuracy: 0.9323
Epoch 8/200
45000/45000 [==============================] - 1s 23us/step - loss: 0.1279 - accuracy: 0.9377
Epoch 9/200
45000/45000 [==============================] - 1s 28us/step - loss: 0.1184 - accuracy: 0.9438
Epoch 10/200
45000/45000 [====================

45000/45000 [==============================] - 1s 22us/step - loss: 0.0277 - accuracy: 0.9927
Epoch 154/200
45000/45000 [==============================] - 1s 22us/step - loss: 0.0283 - accuracy: 0.9926
Epoch 155/200
45000/45000 [==============================] - 1s 22us/step - loss: 0.0279 - accuracy: 0.9924
Epoch 156/200
45000/45000 [==============================] - 1s 21us/step - loss: 0.0288 - accuracy: 0.9925
Epoch 157/200
45000/45000 [==============================] - 1s 22us/step - loss: 0.0276 - accuracy: 0.9927
Epoch 158/200
45000/45000 [==============================] - 1s 22us/step - loss: 0.0278 - accuracy: 0.9927
Epoch 159/200
45000/45000 [==============================] - 1s 22us/step - loss: 0.0274 - accuracy: 0.9927
Epoch 160/200
45000/45000 [==============================] - 1s 23us/step - loss: 0.0277 - accuracy: 0.9926
Epoch 161/200
45000/45000 [==============================] - 1s 22us/step - loss: 0.0283 - accuracy: 0.9926
Epoch 162/200
45000/45000 [===============

## CNN_04

In [15]:
x_data, y_data = x_out_train_denoised, y_out_train_denoised
x_data_test, y_data_test = x_out_test_denoised, y_out_test_denoised

# numpy 행렬로 변환 (행, 열, 1차원)
x_data = x_data.reshape(45000, 3, 1)
x_data_test = x_data_test.reshape(15000, 3, 1)

cnn_04 = Sequential()
cnn_04.add(Conv1D(128,3,activation='sigmoid', input_shape = (3,1), padding = 'same', strides = 1))
cnn_04.add(GlobalAveragePooling1D())  # 각 차원마다 평균값 도출
cnn_04.add(Dropout(0.5))  # 비활성화(0.1), 활성화(0.9)
cnn_04.add(Dense(10, activation = 'sigmoid'))  # 신경망 비슷

# CNN training
cnn_04.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
cnn_04.fit(x_data, y_data, batch_size = 100, epochs = 200)

train_result = cnn_04.evaluate(x_data, y_data, batch_size = 1000, verbose = 1)
test_result = cnn_04.evaluate(x_data_test, y_data_test, batch_size = 1000, verbose = 1)

# Save model
cnn_04.save('cnn_04.h5')

# CNN accuracy
print("[CNN] Training data accuracy: {:.4f}".format(train_result[1]))
print("[CNN] Testing data accuracy: {:.4f}".format(test_result[1]))

Epoch 1/200
45000/45000 [==============================] - 1s 21us/step - loss: 0.3016 - accuracy: 0.8965
Epoch 2/200
45000/45000 [==============================] - 1s 18us/step - loss: 0.2302 - accuracy: 0.9016
Epoch 3/200
45000/45000 [==============================] - 1s 17us/step - loss: 0.2037 - accuracy: 0.9048
Epoch 4/200
45000/45000 [==============================] - 1s 17us/step - loss: 0.1857 - accuracy: 0.9098
Epoch 5/200
45000/45000 [==============================] - 1s 18us/step - loss: 0.1712 - accuracy: 0.9158
Epoch 6/200
45000/45000 [==============================] - 1s 18us/step - loss: 0.1584 - accuracy: 0.9217
Epoch 7/200
45000/45000 [==============================] - 1s 17us/step - loss: 0.1484 - accuracy: 0.9267
Epoch 8/200
45000/45000 [==============================] - 1s 17us/step - loss: 0.1397 - accuracy: 0.9310
Epoch 9/200
45000/45000 [==============================] - 1s 18us/step - loss: 0.1320 - accuracy: 0.9350
Epoch 10/200
45000/45000 [====================

45000/45000 [==============================] - 1s 17us/step - loss: 0.0371 - accuracy: 0.9845
Epoch 154/200
45000/45000 [==============================] - 1s 18us/step - loss: 0.0369 - accuracy: 0.9845
Epoch 155/200
45000/45000 [==============================] - 1s 17us/step - loss: 0.0376 - accuracy: 0.9841
Epoch 156/200
45000/45000 [==============================] - 1s 19us/step - loss: 0.0370 - accuracy: 0.9845
Epoch 157/200
45000/45000 [==============================] - 1s 20us/step - loss: 0.0370 - accuracy: 0.9846
Epoch 158/200
45000/45000 [==============================] - 1s 19us/step - loss: 0.0376 - accuracy: 0.9841
Epoch 159/200
45000/45000 [==============================] - 1s 19us/step - loss: 0.0372 - accuracy: 0.9843
Epoch 160/200
45000/45000 [==============================] - 1s 20us/step - loss: 0.0371 - accuracy: 0.9846
Epoch 161/200
45000/45000 [==============================] - 1s 20us/step - loss: 0.0372 - accuracy: 0.9843
Epoch 162/200
45000/45000 [===============